명사 리스트를 (명사, 빈도수) 리스트로 변환


In [1]:

def get_word_count_pair_list(word_list: list): # get_word_count_pair_list(['apple', 'apple']) == [('apple', 2)]
    word_list = sorted(word_list)
    result = []

    start = 0
    end = 0
    while start < len(word_list):
        while end < len(word_list) and word_list[start] == word_list[end]:
            end = end + 1
        result.append((word_list[start], end - start))
        start = end
    return result


극성 사전 불러오기

In [2]:
from common import load_polar_scores

polar_scores = load_polar_scores()
print(polar_scores)

{'의기': 4, '영웅': 5, '신념': 5, '용기': 5, '희생정신': 5, '희생': 5, '죽음': 1, '절망': 1, '금지': 1, '빌미': 2, '비화': 2, '응원': 5, '책임': 2, '무모': 1, '애국': 4, '훌륭': 5, '곤란': 2, '이기적': 1, '부족': 1, '대단': 5, '불행': 1, '마땅': 4, '환영': 4, '죄송': 2, '욕': 1, '인정': 5, '무사귀환': 5, '불의': 2, '무시': 3, '범죄행위': 1, '영웅적행동': 5, '비난': 1, '욕심': 1, '진심': 4, '기원': 5, '도움': 5, '존경': 5, '위용': 4, '승리': 4, '평화': 4, '열망': 4, '소신': 3, '외교분쟁': 1, '걱정': 4, '숭고': 5, '인류애': 4, '비겁': 2, '배신자': 1, '수치': 1, '민폐': 1, '자중': 2, '변명': 2, '망신': 1, '단결': 4, '요란': 2, '실망': 1, '박탈': 1, '외교문제': 1, '해': 1, '위험': 2, '머저리': 1, '심각': 2, '비판': 1, '살벌': 2, '허세': 2, '개': 1, '걸림돌': 2, '수호': 4, '무작정': 2, '처벌': 1, '문제': 3, '천운': 4, '고발': 1, '자유': 5, '몸조심': 5, '자랑': 1, '파견': 2, '싸움': 1, '이미지': 1, '지': 1, '도둑': 1, '건강': 4, '조심': 5, '까는놈': 4}


평가함수 정의

In [3]:
from __future__ import annotations

from common import combi2, combi3, nouns


def evaluate(comment: str) -> float:
    word_list = comment.split(' ')
    noun_list = []
    for i in range(len(word_list)):
        if word_list[i] in nouns:
            noun_list.append(word_list[i])
        elif combi2(word_list[i]) != []:
            t = combi2(word_list[i])
            for j in range(len(t)):
                noun_list.append(t[j])
        elif combi3(word_list[i]) != []:
            t = combi3(word_list[i])
            for j in range(len(t)):
                noun_list.append(t[j])
    c = get_word_count_pair_list(noun_list)

    num_words = 0
    sum_of_product = 0
    for i in range(len(c)):
        word, frequency = c[i]
        num_words += frequency
        score = polar_scores.get(word, 0)
        sum_of_product += score * frequency

    if num_words == 0:
        # 댓글에서 명사를 발견 못한 경우
        return -1

    return sum_of_product/num_words


뉴스 댓글 불러오기

In [4]:
from common import read_from_csv

rows = read_from_csv('test.csv')
print(rows[:3])


[('N', '개소리도 작작하자. 영화나 드라마를 너무 많이 봤네.. 도와주고 싶으면 기부를 해라.. 지가 무슨 슈퍼맨이야? 제발 생각좀 하고 행동하자.. 넌 주민등록번호가 있는 이상 한국인이야. 국가가 너를 보호할 의무가 있다고. 가서 피랍되거나 포로가 되서 외교문제 골치썩고 너같은 놈들 떼거지로 가면 나라입장에서도 곤란하다고. 우리나라는 저기 서유럽 독일 영국 프랑스처럼 러시아랑 떨어져있는게 아니고 바로 옆에 붙어있다고. 강대국 사이에 껴있어 어정쩡하게 가면 족도밥도 안되는게 우리나란데.. 지혜롭게 생각하자 그게 최선이었나'), ('N', '갈꺼면 조용히 가라. 홍보 오지네. 성추행 전적 벗을라고 애쓴다.'), ('N', '자기 목숨 받치는건 훌륭하지만 아니 외교적 일도 생각 안하는 건 좀')]


댓글 평가

In [5]:
matched = 0  # 예측과 레이블이 일치하는 댓글의 갯수
pos = 0 # 함수에 의해 긍정적이라고 평가된 댓글의 갯수
neg = 0 # 함수에 의해 부정적이라고 평가된 댓글의 갯수

for row in rows:
    label, text = row
    score = evaluate(text)

    THRESHOLD = 0.4
    predict = 'P' if score >= THRESHOLD else 'N'

    if label == predict:
        print(f'correct: {text}')
        if predict == 'P':
            pos += 1
        else:
            neg += 1
        matched += 1
    else:
        print(f'incorrect: {text}')
        if predict == 'P':
            pos += 1
        else:
            neg += 1


correct: 개소리도 작작하자. 영화나 드라마를 너무 많이 봤네.. 도와주고 싶으면 기부를 해라.. 지가 무슨 슈퍼맨이야? 제발 생각좀 하고 행동하자.. 넌 주민등록번호가 있는 이상 한국인이야. 국가가 너를 보호할 의무가 있다고. 가서 피랍되거나 포로가 되서 외교문제 골치썩고 너같은 놈들 떼거지로 가면 나라입장에서도 곤란하다고. 우리나라는 저기 서유럽 독일 영국 프랑스처럼 러시아랑 떨어져있는게 아니고 바로 옆에 붙어있다고. 강대국 사이에 껴있어 어정쩡하게 가면 족도밥도 안되는게 우리나란데.. 지혜롭게 생각하자 그게 최선이었나
correct: 갈꺼면 조용히 가라. 홍보 오지네. 성추행 전적 벗을라고 애쓴다.
correct: 자기 목숨 받치는건 훌륭하지만 아니 외교적 일도 생각 안하는 건 좀
correct: 625전쟁때는 천운으로 유엔군 도움 받았지만 지금 우크라이나는 불행히도 유엔안보리(러시아)때문에 참전도움 못받으니까 이케 비공식적으로 원조하는것은 지극히 마땅하다!!!누가 이근의 영웅적행동을 감히 비난하는가??
correct: 만약 피랍되도 개인의 일탈이라는 명분이 확실하게 있으니 외교문제로 확장시키긴 러시아 입장에서도 넌센스 여튼저튼 한국군인의 타이틀을 처음 가지고 합류한것이니 인정받고 오시길
correct: 좋게만 볼 일은 아닐듯. 전쟁 끝난 후를 생각했을 때 서방국가들과 중러 사이에 낀 우리나라는 한쪽으로 기울면 당하기 쉽다. 후방지원 정도를 하고 왔으면 하는 바람이다.
incorrect: 정답은 없는 것 같다. 하지만 어려운 결정임은 틀림이 없다. 불의를 보고 가만히 보고만 있으면 죄짓는 것 같아 참여를 결정한 것으로 보이고 국가 통제에 따르지 않는 것에 대하여 여러문제가 예견된 수 있다. 하지만 임이 떠난 상태로 돌아올 땐 무사히 건강하게 돌아오길 기원한다.
correct: 자유를 위해 평화를위해 목숨을건 당신들의 결단에 경의를 표합니다 우리도 북한의 침공으로 어려울때 미국을 비롯해 세계 각국의 도움을 받아 나라를 지컸기에 오늘의 대한민국이 있었습

실행 결과 출력

In [6]:
print('정확도:', matched / len(rows))
print('총 댓글 갯수: ', len(rows))
print('긍정, 부정 퍼센트:', pos/len(rows) * 100, neg/len(rows) * 100)

정확도: 0.9
총 댓글 갯수:  20
긍정, 부정 퍼센트: 35.0 65.0
